# Finding hyperbolic Trajectory
----
We take as input the model paramters, forcing type and parameters, time interval and number of points along the time interval we would like to evaluate the hyperbolic trajectories at.

In [1]:

import numpy as np
from numpy import cos, sin, cosh, sinh, tanh, array,pi, exp,array,sqrt
from numpy.linalg import norm,solve
import matplotlib.pyplot as plt
# from scipy.integrate import solve_ivp
import sdeint
import scipy.linalg
import scipy.io
from model import linearisedSystem,model,eigen,Usolve,Lsolve

In [2]:
# Flow function, aim is to solve dY = f(Y,t)dt + G(Y,t)dW on [0,dt]
def phi(dt,y0,f,G,dW):
    tspan = np.linspace(0,dt,len(dW) + 1)    
    result = sdeint.itoSRI2(f, G, y0, tspan,dW = dW)

    return result[-1]

In [30]:
# Define initial paramters
T = 10
N = 51
t = np.linspace(-T,T,N)
# parametrs
k = 1 # damping coefficient
D = lambda v : k*v #linear damping
h = 1 # potential parameter, need h>0
forcingtype, variance = "filtered-white-noise", 0.5
seednumber = 123456789
sde_resolution = 100
dW = sdeint.deltaW(sde_resolution*N,2,2*T/(sde_resolution*N))
def G(x,t):
    return np.array([[0,0],[0,0],[variance,0],[0,variance]])

f,Aplus, Aminus, xplus, xminus = model(h,k,D)    # the vector field, matrices for linearised systems at equilibria, and equilibria
dt = 2*T/N
Pplus,Pplusinv,Pminus,Pinvminus,lambda2, omega, lambda3, lambda4  =  eigen(h,k)

epsF = 5e-5
epsC = 5e-4

In [77]:
A = Aplus
P = Pplus
Pinv = Pplusinv
x0 =xplus
X0 = np.zeros((len(t),4))

In [36]:
A = Aminus
P = Pminus
Pinv = Pinvminus
x0 =xminus
X0 = np.zeros((len(t),4))

In [41]:
def F(X,N):
    # N number of points taken, X is diplacement from x0
    Y = np.zeros((N,4))
    X = X.reshape((N,4))
    for i in range(0, N-1):
        Y[i] = phi(dt,X[i]+x0,f,G,dW[sde_resolution*i:(sde_resolution*i+sde_resolution)]) - x0  - X[i+1]
        # Y[i] = phi(X[i] + x0,t[i]) - X[i+1] - x0   # ensures this is a trajectory.
    # Boundary conditions (stable/unstable coordinates are zero to keep bounded ( need displacement from equilibrium)
    Y[-1][1] = Pinv.dot(X[0])[0]
    Y[-1][1] = Pinv.dot(X[0])[1]
    Y[-1][2] = Pinv.dot(X[0])[2]
    Y[-1][3] = Pinv.dot(X[-1])[3] # unstable
    return Y.reshape(4*N)
Dphi = scipy.linalg.expm(dt*A) # matrix exponential for linear flow displacement from x0
# Derivative for linear system
DF0 = np.zeros((4*N,4*N))
for i in range(0,N-1):
    # find linear derivative
    DF0[4*i:4*i + 4, 4*i:4*i + 4] = Dphi
    DF0[4*i:4*i +4,4*i+4:4*i+8] = -np.eye(4)
DF0[4*N-4,0:4] = np.dot(Pinv.transpose(),array([1,0,0,0]))
DF0[4*N-3,0:4] = np.dot(Pinv.transpose(),array([0,1,0,0]))
DF0[4*N-2,0:4] = np.dot(Pinv.transpose(),array([0,0,1,0]))
DF0[4*N-1,4*N-4:] = np.dot(Pinv.transpose(),array([0,0,0,1]))
Permute, L, U = scipy.linalg.lu(DF0) # numerical LU decomposition. This is computed once to maximise efficiency
def DF0solve(y0):   # we only store the solution function to DF*h = y.
    x0 = np.transpose(Permute).dot(y0)
    return Usolve(Lsolve(x0,L),U)

In [42]:
X = X0.reshape(-1)
Fx = F(X0,N)
delta = DF0solve(Fx)
for _ in range(0,15):
    X = X - delta
    Fx = F(X,N)
    delta = DF0solve(Fx)
    norm_Fx = np.linalg.norm(Fx)
    print(f'F_error = {norm_Fx}, x_error = {np.linalg.norm(delta)}')
    if norm_Fx > 1000 or (norm_Fx < epsF and np.linalg.norm(delta)< epsC):
        break
X_hyp = X.reshape((len(t),4)) + x0

F_error = 0.16230578269763832, x_error = 0.3404754784873867
F_error = 0.03382231486582066, x_error = 0.08594994143230274
F_error = 0.005817871811176807, x_error = 0.015035946487217719
F_error = 0.0010484353439124128, x_error = 0.002281889360379466
F_error = 0.00029709750299504467, x_error = 0.0005957956495498256
F_error = 9.128433347596219e-05, x_error = 0.0001925868458127448
F_error = 2.559714614414712e-05, x_error = 5.600282598482333e-05


In [39]:
X_hypminus = X_hyp

In [43]:
%matplotlib qt
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(projection='3d')
ax.plot(X_hyp[:,0],X_hyp[:,1], t,label = 'positive trajectory')
ax.plot(X_hypminus[:,0],X_hypminus[:,1], t,label = 'negative trajectory')
ax.legend()
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('t')
plt.show()

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [78]:
def F_S(X,q,j,imin,M):
    # N number of points taken, X is diplacement from x0
    Y = np.zeros((M,4))
    X = X.reshape((M,4))
    for i in range(0, M-1):
        Y[i] = phi(dt,X[i]+X_hyp[imin+i],f,G,dW[sde_resolution*i:(sde_resolution*i+sde_resolution)]) - X_hyp[imin+i+1]  - X[i+1]
        # Y[i] = phi(X[i] + x0,t[i]) - X[i+1] - x0   # ensures this is a trajectory.
    # Boundary conditions (stable/unstable coordinates are zero to keep bounded ( need displacement from equilibrium)
    Y[-1][1] = Pinv.dot(X[j-imin])[0] - q[0]
    Y[-1][1] = Pinv.dot(X[j-imin])[1] -q[1]
    Y[-1][2] = Pinv.dot(X[j-imin])[2] - q[2]
    Y[-1][3] = Pinv.dot(X[-1])[3] # unstable
    return Y.reshape(4*M)
def get_DF0_Ssolve(j,imin,M):
    DF0_S = np.zeros((4*M,4*M))
    j = j-imin
    for i in range(0,M-1):
        # find linear derivative
        DF0_S[4*i:4*i + 4, 4*i:4*i + 4] = Dphi
        DF0_S[4*i:4*i +4,4*i+4:4*i+8] = -np.eye(4)
    DF0_S[4*M-4,4*j:4*j+4] = np.dot(Pinv.transpose(),array([1,0,0,0]))
    DF0_S[4*M-3,4*j:4*j+4] = np.dot(Pinv.transpose(),array([0,1,0,0]))
    DF0_S[4*M-2,4*j:4*j+4] = np.dot(Pinv.transpose(),array([0,0,1,0]))
    DF0_S[4*M-1,4*M-4:] = np.dot(Pinv.transpose(),array([0,0,0,1]))
    Permute, L, U = scipy.linalg.lu(DF0_S) # numerical LU decomposition. This is computed once to maximise efficiency
    def DF0_Ssolve(y0):   # we only store the solution function to DF*h = y.
        x0 = np.transpose(Permute).dot(y0)
        return Usolve(Lsolve(x0,L),U)
    return DF0_Ssolve

In [79]:
Deltatminus = int(-np.log(10)/(dt*lambda3))
Deltatplus = int(np.log(10)/(dt*lambda4))

In [80]:
def Find_stable_trajectory(q,j,epsF =1e-5,epsC = 1e-5):
# if __name__ == '__main__':    

    Deltatminus = int(-np.log(10)/(dt*lambda3))
    Deltatplus = int(np.log(10)/(dt*lambda4))
    imin = j -Deltatminus
    imax = j+ Deltatplus
    M = imax -imin + 1
    p_0 = np.matmul(P,np.hstack((q,0)))
    S = np.arange(imin,imax+1,1) - j
    X0 = [np.matmul(scipy.linalg.expm(s*dt*A),p_0) for s in S]
    X0 = np.array(X0)
    X = X0.reshape(-1)
    Fx = F_S(X,q,j,imin,M)
    DF0_Ssolve = get_DF0_Ssolve(j,imin,M)
    delta = DF0_Ssolve(Fx)
    for _ in range(0,100):
        X = X - delta
        Fx = F_S(X,q,j,imin,M)
        delta = DF0_Ssolve(Fx)
        norm_Fx = np.linalg.norm(Fx)
        print(f'F_error = {norm_Fx}, x_error = {np.linalg.norm(delta)}')
        if norm_Fx > 1000 or (norm_Fx < epsF and np.linalg.norm(delta)< epsC):
            break
    if (norm_Fx < epsF and np.linalg.norm(delta)< epsC):
        X_S = X.reshape((M,4)) + X_hyp[imin:imax+1]
        flag = True
    else:
        X_S = X0
        flag = False
    return X_S[j-imin],flag

In [86]:
M = 10
R = 1.5
q1,q2,q3 = np.meshgrid(np.linspace(-R,R,M),np.linspace(-R,R,M),np.linspace(-R,R,M))
q1=q1.reshape(-1)
q2=q2.reshape(-1)
q3 = q3.reshape(-1)
Q = np.vstack((q1,q2,q3)).T

In [87]:
import joblib
from joblib import Parallel, delayed


j = 25
print(f'Finding stable manifold on t = {t[j]}')
def get_point(q):
    X_S,flag = Find_stable_trajectory(q,j)
    p = X_S
    return p,flag
SM = []
for q in Q:
    SM.append(get_point(q)) 
# SM = Parallel(n_jobs=2)(delayed(get_point)(q) for q in Q)
P = [p[0] for p in SM]


Finding stable manifold on t = 0.0
F_error = 23.712448205917763, x_error = 37.20382682402138
F_error = 13.41996838433187, x_error = 16.159424827567648
F_error = 14.030158185988384, x_error = 22.1623137790983
F_error = 15.854526384451198, x_error = 21.582899669682334
F_error = 1.7359536292281612, x_error = 2.8698673749894454
F_error = 6.076165924453877, x_error = 8.844649968157018
F_error = 2.7566369152076766, x_error = 3.6557956812403183
F_error = 1.7304895634993125, x_error = 2.726974679103972
F_error = 2.3432254539934907, x_error = 3.309108762749248
F_error = 0.31951583148488083, x_error = 0.3163194951697895
F_error = 0.9932123312616656, x_error = 1.4685461899114665
F_error = 0.618692393696332, x_error = 0.8419223800014642
F_error = 0.19184479969340082, x_error = 0.31557701747954175
F_error = 0.40044707052412243, x_error = 0.5746922886135973
F_error = 0.11066992889068557, x_error = 0.1345044485078247
F_error = 0.145555900110845, x_error = 0.21964672892482995
F_error = 0.1243009763196

c:\Users\alexm\OneDrive\Documents\GitHub\ship-capsize-2022\Final code\model.py:27: RuntimeWarning: overflow encountered in scalar power
  f = lambda y,t : array([y[2] ,y[3] ,-h*(y[0]-y[1]**2) - kx*y[2] ,-y[1] + y[0]*y[1] - D(y[3])])
c:\Users\alexm\OneDrive\Documents\GitHub\ship-capsize-2022\Final code\model.py:27: RuntimeWarning: overflow encountered in scalar multiply
  f = lambda y,t : array([y[2] ,y[3] ,-h*(y[0]-y[1]**2) - kx*y[2] ,-y[1] + y[0]*y[1] - D(y[3])])
c:\Users\alexm\OneDrive\Documents\GitHub\ship-capsize-2022\Final code\model.py:27: RuntimeWarning: invalid value encountered in scalar subtract
  f = lambda y,t : array([y[2] ,y[3] ,-h*(y[0]-y[1]**2) - kx*y[2] ,-y[1] + y[0]*y[1] - D(y[3])])
c:\Users\alexm\OneDrive\Documents\GitHub\ship-capsize-2022\Final code\model.py:27: RuntimeWarning: invalid value encountered in scalar add
  f = lambda y,t : array([y[2] ,y[3] ,-h*(y[0]-y[1]**2) - kx*y[2] ,-y[1] + y[0]*y[1] - D(y[3])])


F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan, x_error = nan
F_error = nan,

In [ ]:

SM = np.array(P)
np.save('SDE_SM.npy',SM)
np.save('SDE_q.npy',Q)

In [62]:
v_y = scipy.interpolate.Rbf(SM[:,0],SM[:,1],SM[:,2],SM[:,3])

In [69]:
X,VX = np.meshgrid(np.linspace(-3,3,20),np.linspace(-3,3,20))
VY = v_y(X,0*X,VX)

In [ ]:
omega

In [71]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot_surface(X,VX,VY)
ax.set_xlabel('x')
ax.set_ylabel('v_x')
ax.set_zlabel('v_y')
plt.show()